In [1]:
#######################
# Project 3
# ESE 572
# Layer 2 Channel Coding
#######################

# Choose a probability of bit error to use when simulating the receiving end.

prob = 0.1
# prob = 0.01

In [2]:
#######################
# Step 1
#######################

# Function to convert string to binary using ASCII encoding
def string_to_binary(string):
    binary = ''
    binaryS = ''
    binaryC = ''
    counter = 0
    for char in string:
        ascii_code = ord(char) # Get ASCII code of character
        if counter == 24:
            binary += format(ascii_code, '08b')
            binaryS += format(ascii_code, '08b') + ' '
            binaryC += format(ascii_code, '08b') + '\n'
            counter = 0
        else:
            binary += format(ascii_code, '08b')
            binaryS += format(ascii_code, '08b') + ' '
            binaryC += format(ascii_code, '08b')
            counter += 1
    return binary, binaryS, binaryC

# Load the document
filename = 'input.txt'
text = ''
with open(filename,'r') as i:
    text += i.readline()
    
binary_data, binaryS_data, binaryC_data = string_to_binary(text)

# binary_data is single line
# binaryS_data contain spaces to delimitate each character
# binaryC_data is split as chunks of 200 bits


In [3]:
#######################
# Step 2
#######################

# Create CRC for g(D) = [D16 + D12 + D5 + 1]
crc_str = '10001000000100001' # divisor
int_crc_str = int(crc_str,2)
frames = []

binary_chunks = binaryC_data.split('\n')
for chunk in binary_chunks[:-1]:
    temp = chunk + "0000000000000000"
    temp = int(temp,2)
    crc = temp % int_crc_str
    crc = format(crc, '016b')
    if len(crc) > 16:
        crc = crc[-16:]
    frames.append(chunk + crc)

print((frames[0]))

010010010110111000100000001100010011100000110101001100110010110000100000011100000111001001101111011011010110100101101110011001010110111001110100001000000101001101110100001011100010000001001100011011111111100010111010


In [4]:
#######################
# Step 3
#######################
## Channel Coding **NEW**

import numpy as np

def encode_message(msg, matrix):
    code = np.mod(np.dot(msg, matrix), 2)
    return code

def decode_message(code, valid_codewords):
    incorrect_vec = []
    incorrect = 0
    for codeword in valid_codewords:
        for ind in range(len(codeword)):
            if codeword[ind] != code[ind]:
                incorrect += 1
        incorrect_vec.append(incorrect)
        incorrect = 0
    closest = np.min(incorrect_vec)
    closest = np.where(incorrect_vec == closest)[0][0]
    decoded_msg = valid_data[closest]
    return decoded_msg

G = np.array([[1,1,1,1,1,1,1,1],
              [0,0,0,0,1,1,1,1],
              [0,0,1,1,0,0,1,1],
              [0,1,0,1,0,1,0,1]])

valid_data = [[0,0,0,0], [0,0,0,1], [0,0,1,0], [0,0,1,1],
              [0,1,0,0], [0,1,0,1], [0,1,1,0], [0,1,1,1],
              [1,0,0,0], [1,0,0,1], [1,0,1,0], [1,0,1,1],
              [1,1,0,0], [1,1,0,1], [1,1,1,0], [1,1,1,1]]

valid_codewords = []
for data in valid_data:
    valid_codewords.append(encode_message(data,G))
print(valid_codewords)

############ Example ############
msg = np.array([0, 1, 1, 1])
print("Sent Message:", msg)
print()

code = encode_message(msg, G)
print("Encoded code:", code)
print()

decoded_msg = decode_message(code, valid_codewords)
print("Decoded message:", decoded_msg)
############ End Example ############

def array_to_string(array):
    output = ""
    for each in array:
        output += str(each)
    return output

def string_to_array(string):
    output = []
    for each in string:
        output.append(int(each))
    output = np.array(output)
    return output

def RM_string(input_string):
    output_string = ""
    ind = 0
    while(ind <= len(input_string)):
        dat = input_string[ind:ind+4]
        data_string = string_to_array(dat)
        if data_string.size != 0:
            data_string = data_string.reshape((1,4))
            encoded = encode_message(data_string,G)[0]
            output_string += array_to_string(encoded)
        else:
            output_string += str(dat[-1:])
        ind += 4
    return output_string

binaryRM = []
for frame in frames:
    binaryRM.append(RM_string(frame))
    
print(len(binaryRM[0]))

[array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 1, 0, 1, 0, 1, 0, 1]), array([0, 0, 1, 1, 0, 0, 1, 1]), array([0, 1, 1, 0, 0, 1, 1, 0]), array([0, 0, 0, 0, 1, 1, 1, 1]), array([0, 1, 0, 1, 1, 0, 1, 0]), array([0, 0, 1, 1, 1, 1, 0, 0]), array([0, 1, 1, 0, 1, 0, 0, 1]), array([1, 1, 1, 1, 1, 1, 1, 1]), array([1, 0, 1, 0, 1, 0, 1, 0]), array([1, 1, 0, 0, 1, 1, 0, 0]), array([1, 0, 0, 1, 1, 0, 0, 1]), array([1, 1, 1, 1, 0, 0, 0, 0]), array([1, 0, 1, 0, 0, 1, 0, 1]), array([1, 1, 0, 0, 0, 0, 1, 1]), array([1, 0, 0, 1, 0, 1, 1, 0])]
Sent Message: [0 1 1 1]

Encoded code: [0 1 1 0 1 0 0 1]

Decoded message: [0, 1, 1, 1]
432


In [5]:
#######################
# Step 4
#######################

# flag = '01111110'
flag = '000111111111111111111000' # tripled
framed_frames = []
bitstuff=0
for binRM in binaryRM:
    i = 0
    
    while(i <= (len(binRM) - 18)):
        if binRM[i:i+18] == '111111111111111111':
            binRM = binRM[:i+17] + '0' + binRM[i+17:] # bit stuff after any string of five 1s
            bitstuff+=1
            i += 18
        else:
            i += 1
    framed_frames.append(flag + binRM)
    
framed_frames.append(flag) # after last FEC frame, insert another flag
print('bits stuffed: '+str(bitstuff))
print(framed_frames[0])

bits stuffed: 0
000111111111111111111000000011111010101000111100110000110011001100000000011001100101010101100110111111110110011001011010011001100110011000110011111100000011001100000000011010010000000001101001001100110011110010010110001111001010010100111100101010100011110011000011001111000101101000111100110000110110100100001111001100110000000001011010011001100110100100001111001100111100001100110011000000000000111111110000001111001001011010010110111111111001100111001100


In [6]:
#######################
# Sequence to Transmit
#######################

sequence = ''
for fram in framed_frames:
    sequence += fram
    # print(len(fram))
    
print(len(sequence))

20544


In [7]:
#######################
# Step 5: Error
#######################

import random

lim = {0.1: 9,
       0.01: 99}

rxbits = ''
errors = 0

for bit in sequence:
    rnum = random.randint(0,lim[prob])
    # rnum = 1 # no errors
    if rnum == 0: # error bit
        errors += 1
        if bit == '1':
            bit = '0'
        else:
            bit = '1'
    rxbits += bit
print("Number added random errors:", errors)
print(len(rxbits))

Number added random errors: 2077
20544


In [8]:
#######################
# Step 5: Unstuffing
#######################

def unstuff(bits):
    i = 0
    while i < len(bits)-3:
        #print(bits[i:i+19])
        if bits[i:i+19] == '1111111111111111101':
            print(bits[i:i+19])
            bits = bits[:i+17] + bits[i+18:] # remove bit stuffing
            print(bits[i:i+19])
            i += 18
        else:
            i += 1
    return bits

rxbits = unstuff(rxbits)
print(len(rxbits))

20544


In [9]:
#######################
# Step 5: Fixing
#######################

corrected_bits = rxbits
# finding flags to reseparate frames
i = 0
start = -1
end = -1
new_frames = []
flag_flag = 0

while (i < len(corrected_bits)):
    if corrected_bits[i:i+24] == flag:
        flag_flag = 1
        if start != -1:
            end = i
            temp = corrected_bits[start:end]
            new_frames.append(temp)
            start = i
        else:
            start = i
    if flag_flag:
        i += 24
        flag_flag = 0
    else:
        i += 1

new_frames.append(corrected_bits[start:])

new_frames = new_frames[:-1]
print(new_frames[0])
print(len(new_frames[-2]))

# frames separated, unstuffed but need :: untripled, CRC checked, and then converted to text

0001111111111111111110001000111110101010001111001100001100110011000000000010011001010100001001101111001101100110110110110110010001100110001100111011001110111011000000000110100100000000011000010011001110111101010101100001110010100101001111101010111000111100010000110011110001011010010011001100001101111000000011100011001100100000010110100110011101101101000010110011001111100101001100011000000000001111111110000011110010010101100100001111110110011101110011000001111011111011110110000110000101001010001111001010111001101011011011110011001100000000001111001010010110111110010111100110101100111011001011000100111000111100111111110011100101010100001111001100001101101001010011110011001110000010110110100110100100011100010101010100100110001010001111001010010100111100010101010011100010000011001100110000000000001111011011101110110101110010011110001001111001001001111010010011111100000000001111101010010111110010010110100001110111111111011110000011110001011101101111001100001100111100000001100011001100000000

In [10]:
#######################
# Step 5: Remove flag
#######################

rxseqs = []
for nf in new_frames:
    rxseqs.append(nf[24:])

print(len(rxseqs[0]))
# need :: CRC checked, and then converted to text

1344


In [11]:
#######################
# Step 5: Decode RM
#######################

temp_rxseqs = []
for frame in rxseqs:
    # frame = frame[:-8]
    temp_frame = ""
    count = 0
    while(count < len(frame)):
        message = string_to_array(frame[count:count+8])
        decoded_msg = decode_message(message, valid_codewords)
        temp_frame += array_to_string(decoded_msg)
        count += 8
    temp_frame += frame[count:]
    temp_rxseqs.append(temp_frame)
    print(temp_frame)

rxseqs = temp_rxseqs
# print(rxseqs)
print(len(rxseqs[0]))

010010010110111000100000001100010011001000110101001100110010001000100000011100000111001001100001011011010110100101101110011001011010111001010100001000000101001101110100001011010010000001001100011000010000100010111010010010000101011101010110100101110011001000000110110101100101011100100110001101101000001000010110111001110100001000000101011101100001011110010110110101100001011011100010000001000011011100100101111101110111001000000110110111000101000110000101011000010110111001100000001000000110100001101001011100010010001000000000011000010010001101110100011011110111001000101100001000000101011101101001000000010110001101101001011000010110110100100000010001110101111000010000
011100100110000101100101011011101000011100110101011000010110011000100000010001010110110001101001011011110111010000100000010010100010000010010001001011000010000001100011011011110110111001100011011001011001000001100101010010001100011100100010111001100101011001000010000001100001011000100110111100010110011101000010000010000100000

In [12]:
#######################
# Step 5: Check CRC
#######################

# Create CRC for g(D) = [D16 + D12 + D5 + 1]
crc_str = '10001000000100001' # divisor
int_crc_str = int(crc_str,2)
failed = 0
total = 0

for rxs in rxseqs:
    crc_rx = rxs[-16:]
    beginning = rxs[:-16] + "0000000000000000"
    beginning = int(beginning,2)
    crc = beginning % int_crc_str
    crc = format(crc,'016b')
    if len(crc) > 16:
        crc = crc[-16:]
    if crc != crc_rx:
        print("crc: "+str(crc)+" crc_rx: "+str(crc_rx))
        failed += 1
    total += 1

print('Number of Failed Frames with Error Probability ' + str(prob) + ': --- ' + str(failed) + ' --- \n')
print('Failed Frames: ' + str((failed / total) * 100) + '%' + ' of ' + str(total) + ' total frames recieved.')

crc: 1110100100000111 crc_rx: 0101111000010000
crc: 0111011110100110 crc_rx: 0101100001101110
crc: 0010001101010010 crc_rx: 1101110001010111
crc: 0100110110011000 crc_rx: 1000110001110000
Number of Failed Frames with Error Probability 0.1: --- 4 --- 

Failed Frames: 100.0% of 4 total frames recieved.


In [13]:
#######################
# Convert Back to Text
#######################

all_rx_bits = ''
char_arr = []
paragraph = ''
i = 0

for rxs in rxseqs:
    if len(rxs[:-16]) < 200:
        continue
    else:
        all_rx_bits += rxs[:-16]

while i < len(all_rx_bits)-7:
    char_arr.append(all_rx_bits[i:i+8])
    i += 8

for cb in char_arr:
    character = chr(int(cb,2))
    if (character!='~'):
        paragraph += character

print(paragraph)

In 1253" pramine®T St- LaºHWV2ÖW&6çBvÖâ7%÷rÜQan` hiq" a#tor, Wiciam Graen5af Eliot J , conceeHÇ"æVB&ñgB@RÆ6übà!!3titut?ns of Ghe leaHÆææqFâFRw&×zærÖGvQ|St, led qhb bounding of  g¥HZ1æwFöâV"§f%'6G5 0´t. Loui. @ rb÷ th 0x40uÉ(W1æB S 2Âvb2ö Öpàxradts!faoded into St$pLo¦HwV'1Â&öô7FrFR ÷Aæion  the 5ou`g cIt). WiHFF@W6âæWv6V6³!26&R"'76æræVRB÷"V@V6öâÒ&÷FæG!7BÂG&ÁAbninG cnd basic gene"al c
´Æ÷U'6W" 6&æGV7FVB÷WG66@4D\e µ6 nrmal wori.g houB3Ð/HRâ6òERf'3âVGVFºöæHLl st%p of the yug ashcÄiEfæwF¶âFæ-&Gv2Fò^3staLish !a Evec`ne rogr%(ÆöâÖ7ö&W"#UÂSBáüâdnr the succeedinb decadd7:FÇ2+R3öçFëVFGV6F


In [14]:
# In 1853, prominent St. Louis merchant Wayman Crow and his pastor, William Greenleaf Eliot Jr., concerned about 
# the lack of institutions of higher learning in the growing midwest, led the founding of Washington University  
# in St. Louis. During the 1840s and 50s, waves of immigrants flooded into St. Louis, boosting the population of 
# the young city. With these newcomers came a pressing need for education - both industrial training and basic 
# general courses - conducted outside of normal working hours. So the first educational step of the young 
# Washington University was to establish an evening program on October 22, 1854. Over the succeeding decades, 
# the continuing education program underwent many changes. The university flourished at its location in downtown 
# St. Louis for its first 50 years, growing from an evening program to an institution offering a full slate of 
# scientific, liberal arts and classical course offerings. In time, schools of law and fine arts were added. In 
# 1891, the school acquired the St. Louis Medical College to form a medical department, which merged with the 
# Missouri Medical College in 1899.